# 🧪 Techne Finance - Analytics Laboratory

**Purpose**: Heavy computations for your DeFi bot that VPS can't handle.

## Modules:
1. **APY Trend Analysis** - Linear regression to predict yield decay
2. **Wash Trading Detection** - K-means clustering on swap patterns
3. **Parking Threshold Optimization** - Monte Carlo backtesting

---

In [ ]:
# ==========================================
# 🔧 SETUP - Run this first!
# ==========================================
!pip install supabase pandas numpy scikit-learn requests -q

import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from supabase import create_client
import requests

# 📌 YOUR CREDENTIALS (paste from .env)
SUPABASE_URL = "https://your-project.supabase.co"  # @param {type:"string"}
SUPABASE_KEY = "your-anon-key"  # @param {type:"string"}

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print("✅ Setup complete!")

---
## 📈 Module 1: APY Trend Analysis

Calculates the **slope** of APY over time using Linear Regression:
- Positive slope = APY growing
- Negative slope = APY decaying (danger!)

In [ ]:
def analyze_apy_trend(pool_address: str, days: int = 30) -> dict:
    """
    Fetch APY history from DefiLlama and calculate trend.
    Returns JSON with slope, R², and recommendation.
    """
    # Fetch from DefiLlama API
    url = f"https://yields.llama.fi/chart/{pool_address}"
    response = requests.get(url)
    
    if response.status_code != 200:
        return {"error": f"Failed to fetch data: {response.status_code}"}
    
    data = response.json().get("data", [])
    if len(data) < 7:
        return {"error": "Insufficient data points (need at least 7 days)"}
    
    # Take last N days
    recent = data[-days:] if len(data) >= days else data
    
    # Prepare for regression
    df = pd.DataFrame(recent)
    df['day_index'] = range(len(df))
    
    X = df['day_index'].values.reshape(-1, 1)
    y = df['apy'].values
    
    # Linear Regression
    model = LinearRegression()
    model.fit(X, y)
    
    slope = model.coef_[0]  # APY change per day
    r_squared = model.score(X, y)
    current_apy = y[-1]
    avg_apy = np.mean(y)
    volatility = np.std(y)
    
    # Projected APY in 7 days
    projected_7d = current_apy + (slope * 7)
    
    # Recommendation logic
    if slope < -0.5 and r_squared > 0.6:
        recommendation = "AVOID - Strong decay trend"
    elif slope < -0.2:
        recommendation = "CAUTION - Mild decay"
    elif slope > 0.3:
        recommendation = "OPPORTUNITY - Growing yield"
    else:
        recommendation = "STABLE - Flat trend"
    
    return {
        "pool_address": pool_address,
        "analysis_date": datetime.now().isoformat(),
        "days_analyzed": len(recent),
        "current_apy": round(current_apy, 2),
        "avg_apy": round(avg_apy, 2),
        "volatility": round(volatility, 2),
        "slope_per_day": round(slope, 4),
        "r_squared": round(r_squared, 4),
        "projected_apy_7d": round(projected_7d, 2),
        "recommendation": recommendation
    }

# 🧪 TEST - Aerodrome WETH-USDC
result = analyze_apy_trend("747c1d2a-c668-4682-b9f9-296708a3dd90")
print(json.dumps(result, indent=2))

---
## 🔍 Module 2: Wash Trading Detection

Uses **K-Means clustering** on wallet addresses to detect:
- Concentration of volume in few wallets
- Circular trading patterns

**NOTE**: Uses Uniswap V3 Base subgraph (works for most AMMs)

In [ ]:
def detect_wash_trading(pool_address: str) -> dict:
    """
    Analyze swap patterns from The Graph to detect wash trading.
    Returns wash trading probability (0-100%).
    
    Uses Uniswap V3 Base subgraph which has broad coverage.
    """
    # Multiple subgraph options for Base
    SUBGRAPHS = [
        # Uniswap V3 on Base (most active)
        "https://api.studio.thegraph.com/query/48211/uniswap-v3-base/version/latest",
        # Backup: Aerodrome (corrected URL)
        "https://api.studio.thegraph.com/query/50472/aerodrome-slipstream/version/latest"
    ]
    
    # Query that works with standard Uniswap V3 schema
    query = """
    query GetSwaps($pool: String!) {
        swaps(first: 500, where: {pool: $pool}, orderBy: timestamp, orderDirection: desc) {
            origin
            sender
            recipient
            amountUSD
            timestamp
        }
    }
    """
    
    swaps = []
    for graph_url in SUBGRAPHS:
        try:
            response = requests.post(
                graph_url,
                json={"query": query, "variables": {"pool": pool_address.lower()}},
                timeout=10
            )
            if response.status_code == 200:
                data = response.json()
                swaps = data.get("data", {}).get("swaps", [])
                if len(swaps) >= 20:
                    break
        except:
            continue
    
    if len(swaps) < 20:
        return {
            "error": f"Insufficient swaps (found {len(swaps)}, need 20+)",
            "swap_count": len(swaps),
            "tip": "Try a more active pool like WETH-USDC on Uniswap V3"
        }
    
    df = pd.DataFrame(swaps)
    df['amountUSD'] = pd.to_numeric(df['amountUSD'], errors='coerce').fillna(0)
    
    # Use 'origin' or 'sender' depending on what's available
    wallet_col = 'origin' if 'origin' in df.columns and df['origin'].notna().any() else 'sender'
    
    # === METRIC 1: Wallet Concentration ===
    wallet_volumes = df.groupby(wallet_col)['amountUSD'].sum()
    total_volume = wallet_volumes.sum()
    if total_volume > 0:
        top_3_concentration = wallet_volumes.nlargest(3).sum() / total_volume * 100
    else:
        top_3_concentration = 0
    
    # === METRIC 2: Unique Wallets Ratio ===
    unique_wallets = df[wallet_col].nunique()
    wallet_ratio = unique_wallets / len(swaps) * 100
    
    # === METRIC 3: Circular Trading ===
    if 'recipient' in df.columns:
        circular = df[df[wallet_col] == df['recipient']]
        circular_pct = len(circular) / len(swaps) * 100
    else:
        circular_pct = 0
    
    # === METRIC 4: K-Means Clustering on volumes ===
    volumes = df['amountUSD'].values.reshape(-1, 1)
    n_clusters = min(3, len(volumes))  # Don't cluster more than data points
    if n_clusters >= 2:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(volumes)
        cluster_sizes = pd.Series(clusters).value_counts()
        dominant_cluster_pct = cluster_sizes.iloc[0] / len(clusters) * 100
    else:
        dominant_cluster_pct = 100
    
    # === FINAL SCORE ===
    score = (
        (top_3_concentration * 0.4) +
        ((100 - wallet_ratio) * 0.3) +
        (circular_pct * 0.2) +
        (dominant_cluster_pct * 0.1)
    )
    score = min(100, max(0, score))
    
    # Recommendation
    if score >= 70:
        recommendation = "BLOCK - High wash trading probability"
        is_risky = True
    elif score >= 50:
        recommendation = "CAUTION - Moderate concentration"
        is_risky = False
    else:
        recommendation = "SAFE - Organic trading patterns"
        is_risky = False
    
    return {
        "pool_address": pool_address,
        "analysis_date": datetime.now().isoformat(),
        "swaps_analyzed": len(swaps),
        "unique_wallets": unique_wallets,
        "metrics": {
            "top_3_wallet_concentration": round(top_3_concentration, 2),
            "unique_wallet_ratio": round(wallet_ratio, 2),
            "circular_trading_pct": round(circular_pct, 2),
            "dominant_cluster_pct": round(dominant_cluster_pct, 2)
        },
        "wash_trading_score": round(score, 2),
        "is_risky": is_risky,
        "recommendation": recommendation
    }

# 🧪 TEST - Try these known active pools on Base:
# Uniswap V3 WETH-USDC: 0x88A43bbDF9D098eEC7bCEda4e2494615dFd9bB9C
# Uniswap V3 cbETH-WETH: 0x10648BA41B8565907Cfa1496765fA4D95390aa0d

pool = "0x88A43bbDF9D098eEC7bCEda4e2494615dFd9bB9C"  # WETH-USDC Uniswap V3
result = detect_wash_trading(pool)
print(json.dumps(result, indent=2))

---
## 💰 Module 3: Parking Threshold Optimization

**Monte Carlo simulation** to find optimal parking threshold:
- Tests different thresholds ($1k, $5k, $10k, $25k)
- Considers gas costs and Aave yield
- Returns optimal threshold for your capital size

In [ ]:
def optimize_parking_threshold(
    capital_usd: float = 50000,
    gas_cost_usd: float = 11,
    aave_apy: float = 3.5,
    avg_pool_apy: float = 15.0,
    pool_entry_rate: float = 0.6,
    simulation_days: int = 30,
    num_simulations: int = 1000
) -> dict:
    """
    Monte Carlo simulation to find optimal parking threshold.
    """
    thresholds = [1000, 5000, 10000, 25000, 50000]
    results = {}
    
    for threshold in thresholds:
        if threshold > capital_usd:
            continue
            
        total_returns = []
        
        for _ in range(num_simulations):
            balance = capital_usd
            parked = 0
            days_idle = 0
            total_gas = 0
            
            for day in range(simulation_days):
                found_pool = np.random.random() < pool_entry_rate
                
                if found_pool and balance >= 100:
                    if parked > 0:
                        balance += parked
                        parked = 0
                        total_gas += gas_cost_usd
                    
                    daily_yield = balance * (avg_pool_apy / 100 / 365)
                    balance += daily_yield
                    total_gas += gas_cost_usd
                    days_idle = 0
                else:
                    days_idle += 1
                    
                    if days_idle >= 1 and balance >= threshold and parked == 0:
                        parked = balance
                        balance = 0
                        total_gas += gas_cost_usd
                    
                    if parked > 0:
                        daily_aave = parked * (aave_apy / 100 / 365)
                        parked += daily_aave
            
            final = balance + parked - total_gas
            total_returns.append(final)
        
        avg_return = np.mean(total_returns)
        std_return = np.std(total_returns)
        roi = ((avg_return - capital_usd) / capital_usd) * 100
        
        results[threshold] = {
            "avg_final_balance": round(avg_return, 2),
            "std_deviation": round(std_return, 2),
            "roi_pct": round(roi, 2),
            "sharpe_ratio": round(roi / (std_return / capital_usd * 100), 2) if std_return > 0 else 0
        }
    
    optimal = max(results.keys(), key=lambda k: results[k]['sharpe_ratio'])
    
    return {
        "simulation_params": {
            "capital_usd": capital_usd,
            "gas_cost_usd": gas_cost_usd,
            "aave_apy": aave_apy,
            "avg_pool_apy": avg_pool_apy,
            "pool_entry_rate": pool_entry_rate,
            "simulation_days": simulation_days,
            "num_simulations": num_simulations
        },
        "threshold_results": results,
        "optimal_threshold": optimal,
        "optimal_roi": results[optimal]['roi_pct'],
        "recommendation": f"Set MIN_PARKING_AMOUNT_USD = {optimal}"
    }

# 🧪 TEST - Optimize for your capital
result = optimize_parking_threshold(capital_usd=50000, gas_cost_usd=11)
print(json.dumps(result, indent=2))

---
## 📤 Export Results for Claude

Run this cell to generate JSON that you paste into Claude's prompt.

In [ ]:
def generate_claude_report(
    pools_to_analyze: list = None,
    capital: float = 50000
) -> str:
    report = {
        "generated_at": datetime.now().isoformat(),
        "source": "Google Colab Analytics",
        "modules": {}
    }
    
    if pools_to_analyze:
        report["modules"]["apy_trends"] = []
        for pool_id in pools_to_analyze:
            trend = analyze_apy_trend(pool_id)
            report["modules"]["apy_trends"].append(trend)
    
    report["modules"]["parking_optimization"] = optimize_parking_threshold(capital_usd=capital)
    
    parking = report["modules"]["parking_optimization"]
    report["claude_instructions"] = f"""
Based on Monte Carlo simulation ({parking['simulation_params']['num_simulations']} runs):
- Optimal parking threshold: ${parking['optimal_threshold']:,}
- Expected ROI: {parking['optimal_roi']}%

ACTION: Update MIN_PARKING_AMOUNT_USD to {parking['optimal_threshold']}
"""
    
    return json.dumps(report, indent=2)

# 🚀 Generate final report
final_report = generate_claude_report(capital=50000)
print(final_report)

with open('techne_analytics_report.json', 'w') as f:
    f.write(final_report)
print("\n✅ Report saved to techne_analytics_report.json")